note: this notebook is created for understanding the main model function and not meant to be run in this form. This can be run via the other notebooks. 

# Main Model function

`parameters = L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False)`

**description:** Implements a L-layer neural network: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.
    
**input:**
*  X - data, numpy array of shape (num_px * num_px * 3, number of examples)
* Y - true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
* layers_dims - list containing the input size and each layer size, of length (number of layers + 1).
* learning_rate - learning rate of the gradient descent update rule
* num_iterations - number of iterations of the optimization loop
* print_cost - if True, it prints the cost every 100 steps
    
**output:**
* parameters - parameters learnt by the model. They can then be used to predict.


In [ ]:
### CONSTANTS ###
layers_dims = [12288, 20, 7, 5, 1] 
#  4-layer model 
# every element corresponds to the units per layer

In [ ]:
# GRADED FUNCTION: L_layer_model

def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):#lr was 0.009

    np.random.seed(1)
    costs = []                         # keep track of cost
    
    # Parameters initialization. (≈ 1 line of code)

    parameters = initialize_parameters_deep(layers_dims)

    # Loop (gradient descent)
    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        AL, caches = L_model_forward(X, parameters)

        # Compute cost.
        cost = compute_cost(AL, Y)
    
        # Backward propagation.
        grads = L_model_backward(AL, Y, caches)

        # Update parameters.
        parameters = update_parameters(parameters, grads, learning_rate)
                
        # Print the cost every 100 training example
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
            
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

# Initialize parameters W and b

`parameters = initialize_parameters_deep(layers_dims)`

**input:** 
* layer_dims - python array (list) containing the dimensions of each layer in our network

**output:** 
* parameters - python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
  * Wl - weight matrix of shape (layer_dims[l], layer_dims[l-1])
  * bl - bias vector of shape (layer_dims[l], 1)


In [ ]:
# GRADED FUNCTION: initialize_parameters_deep

def initialize_parameters_deep(layer_dims):
    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

# Do Grad Descent for *i* iterations

```
for i in range(0, num_iterations):
```

## Forward Prop

```
AL, caches = L_model_forward(X, parameters)
```

**description:** Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation

**input:** 
* X 
  - train feature instances
  - data, numpy array of shape (input size, number of examples)
* parameters - initialized (from before) for Layer 1 then updated ones per iterations for Layers L-1

**output:**
* AL - activation result from the last layer ($A^{[L]}$)
* caches - contains:
   * linear_cache - A, W, b
   * activation_cache - results of activation

In [ ]:
# GRADED FUNCTION: L_model_forward

def L_model_forward(X, parameters):

    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    # for the L-1 layers
    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation = "relu")
        caches.append(cache)

    
    # for the last layer
    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)], activation = "sigmoid")
    caches.append(cache)

    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

### `linear_activation_forward`

**description:**  
* Implements the forward propagation for the LINEAR->ACTIVATION layer 
* this is done per layer

**function call:** `A, cache = linear_activation_forward(A_prev, W, b, activation)`

**input:**
* A_prev - activations from previous layer (or input data): (size of previous layer, number of examples)
* W - weights matrix: numpy array of shape (size of current layer, size of previous layer)
* b - bias vector, numpy array of shape (size of the current layer, 1)
* activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

**output:**
* A - the output of the activation function, also called the post-activation value 
* cache 
  - a python tuple containing "linear_cache" and "activation_cache"; stored for computing the backward pass efficiently
  - linear_cache contains (A, W, b)
  - activation_cache contains Z which is the result of `linear_forward(A_prev, W, b)`

In [ ]:
def linear_activation_forward(A_prev, W, b, activation):
    
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

#### `linear_forward`

**description:** Implement the linear part of a layer's forward propagation.

**function call:** `Z, cache = linear_forward(A, W, b)`

**input:**
* A - activations from previous layer (or input data): (size of previous layer, number of examples)
* W - weights matrix: numpy array of shape (size of current layer, size of previous layer)
* b - bias vector, numpy array of shape (size of the current layer, 1)

**output:**
* Z - the input of the activation function, also called pre-activation parameter 
* cache - a python tuple containing "A", "W" and "b" ; stored for computing the backward pass efficiently


In [ ]:
def linear_forward(A, W, b):
    
    Z = np.dot(W, A) + b
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

## Compute cost function

`cost = compute_cost(AL, Y)`

**input:**
* AL 
  - activation result from the last layer ($A^{[L]}$) 
  - probability vector corresponding to your label predictions
  - shape is (1, number of examples)

* Y 
  - the target labels
  - true "label" vector (for example: containing 0 if non-cat, 1 if cat)
  - shape is (1, number of examples)

**output:** 
* cost - cross-entropy cost 

In [ ]:
def compute_cost(AL, Y):
    
    m = Y.shape[1]

    # Compute loss from aL and y.
    cost = -(1/m) * np.sum( Y*np.log(AL) + (1-Y)*np.log(1-AL))
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(cost.shape == ())
    
    return cost

## Backprop

` grads = L_model_backward(AL, Y, caches)`

**description:**
* Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group

**input:**
* AL 
  - probability vector, output of the forward propagation (L_model_forward())
* Y 
  - true "label" vector (containing 0 if non-cat, 1 if cat)
* caches 
  - list containing caches per layer

**output:**
*  grads - A dictionary with the gradients
  * grads["dA" + str(l)] = ... 
  * grads["dW" + str(l)] = ...
  * grads["db" + str(l)] = ... 

In [ ]:
def L_model_backward(AL, Y, caches):

    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "dAL, current_cache". Outputs: "grads["dAL-1"], grads["dWL"], grads["dbL"]
    current_cache = caches[L//2]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation = "sigmoid")
    
    # Loop from l=L-2 to l=0
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 1)], current_cache". Outputs: "grads["dA" + str(l)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l +1)], current_cache, activation = "relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

### linear_activation_backward

**description:**
* Implement the backward propagation for the LINEAR->ACTIVATION layer.

**function call:** dA_prev, dW, db = linear_activation_backward(dA, cache, activation)

**input:**
* dA -- post-activation gradient for current layer l 
* cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
* activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

**output:**
* dA_prev - Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
* dW - Gradient of the cost with respect to W (current layer l), same shape as W
* db - Gradient of the cost with respect to b (current layer l), same shape as b

In [ ]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

#### linear_backward

**description:**
* Implement the linear portion of backward propagation for a single layer (layer l)

**function call:** `dA_prev, dW, db = linear_backward(dZ, cache)`

**input:**
* dZ - Gradient of the cost with respect to the linear output (of current layer l)
* cache - tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

**output:**
* dA_prev - Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
* dW - Gradient of the cost with respect to W (current layer l), same shape as W
* db - Gradient of the cost with respect to b (current layer l), same shape as b

In [ ]:
def linear_backward(dZ, cache):

    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = (1/m) * np.dot(dZ, A_prev.T)
    db = (1/m) * np.sum(dZ, axis=1, keepdims = True)
    dA_prev = np.dot(W.T, dZ)
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

## Update Parameters

`parameters = update_parameters(parameters, grads, learning_rate)`

**description:**
Update parameters using gradient descent

**input:**
* parameters - python dictionary containing your parameters 
* grads - python dictionary containing your gradients, output of L_model_backward

**output:**
* parameters - python dictionary containing your updated parameters 
  * parameters["W" + str(l)] = ... 
  * parameters["b" + str(l)] = ...

In [ ]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    for l in range(L):
        parameters["W" + str(l+1)] = parameters['W'+str(l+1)] - learning_rate*grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters['b'+str(l+1)] - learning_rate*grads["db" + str(l+1)]

    return parameters